## Backup
What I want in multithreaded processing:
- async operation of circular buffer.
- actor system.
    - actors can send message concurrently in a thread pool.
    - the message will be queued in FIFO (first-in-first-out) and sequentially processed in each actor.
    - the state will be accessed and modified sequentially.
    - the message queue can be set to be processed with priority.
        - or the message queue can be requeued if the message queue applies FIFO and the current message has to be processed later.
- task parallelism in processing function pipelines inside actors.
    - actor system deals with task (message) dispatch
    - high-performance processing pipeline (by task-oriented programming) executes tasks efficiently.

Before implementing directly on cell search, we first look at a series of simple examples that illustrates the possiblities of realizing the above functionalities.

### Async operation of circular buffer
The circular buffer has read and write functions. After data are read from buffer, they will be processed by a subsequent function named process_data. <br>
To show how async operation works:
- the time for processing data is relatively long (5s). 
- the time for reading data is short (1s).
- the time for writing data is slightly longer than reading (2s)

It should be observed that data are keeping written into the buffer while data are processed in the function.
- note, writing and reading are asynchronous, while reading and processing are sync.

In [4]:
import asyncio
import random
import time

class CircularBuffer:
    def __init__(self, size):
        self.buffer = [None] * size
        self.size = size
        self.write_index = 0
        self.read_index = 0

    def write(self, data):
        """Write data into the circular buffer."""
        self.buffer[self.write_index] = data
        print(f"Written '{data}' at index {self.write_index}")
        self.write_index = (self.write_index + 1) % self.size

    def read(self):
        """Read data from the circular buffer."""
        if self.buffer[self.read_index] is None:
            print(f"No new data at index {self.read_index}.")
            return None
        data = self.buffer[self.read_index]
        print(f"Read '{data}' from index {self.read_index}")
        self.buffer[self.read_index] = None  # Simulate consuming the data
        self.read_index = (self.read_index + 1) % self.size
        return data

def process_data(data):
    """Synchronous function to process the data."""
    print(f"Processing data: {data}")
    # Simulate a long-running synchronous operation
    time.sleep(5)
    print(f"Finished processing data: {data}")

async def write_to_buffer(buffer, interval):
    """Regularly write data into the circular buffer."""
    while True:
        data = random.randint(1, 100)  # Simulate some data to write
        buffer.write(data)
        await asyncio.sleep(interval)  # Simulate periodic writing

async def read_and_process(buffer, interval):
    """Asynchronously read from the circular buffer and process data in a separate thread."""
    loop = asyncio.get_running_loop()  # Get the event loop
    while True:
        data = buffer.read()
        if data is not None:
            # Use run_in_executor to run the process function in a separate thread
            await loop.run_in_executor(None, process_data, data)
        else:
            print("No data to read. Retrying...")
        await asyncio.sleep(interval)  # Simulate asynchronous, delayed reading

async def main():
    buffer_size = 5
    buffer = CircularBuffer(buffer_size)
    
    # Set the interval for reading and writing
    writer_interval = 2  # seconds
    reader_interval = 1  # seconds
    
    # Task to write to the buffer every 2 seconds
    writer_task = asyncio.create_task(write_to_buffer(buffer, writer_interval))
    
    # Task to read from the buffer every 1 second and process synchronously in a separate thread
    reader_task = asyncio.create_task(read_and_process(buffer, reader_interval))
    
    # Run both tasks concurrently, but stop after 50 seconds
    await asyncio.sleep(10)  # Let the tasks run for 50 seconds
    
    # Cancel the tasks after the time limit
    writer_task.cancel()
    reader_task.cancel()
    
    # Wait until the tasks are actually cancelled
    await asyncio.gather(writer_task, reader_task, return_exceptions=True)
    print("Tasks cancelled and program terminated after 10 seconds.")

# Run the program
await main()


Written '31' at index 0
Read '31' from index 0
Processing data: 31
Written '89' at index 1
Written '2' at index 2
Finished processing data: 31
Written '33' at index 3
Read '89' from index 1
Processing data: 89
Written '63' at index 4
Tasks cancelled and program terminated after 10 seconds.


Finished processing data: 89


### Actor system F1: 
Actors send messages concurrently in a thread pool. <br>
Three actors send greetings to each other. <br>
We should see the message that is sent later can be earlier printed in the results. 

In [24]:
import queue
import threading
import time
from concurrent.futures import ThreadPoolExecutor

class Actor:
    def __init__(self, name, actor_system):
        self.name = name
        self.message_queue = queue.Queue()
        self.actor_system = actor_system  # To send messages to other actors
    
    def send_message(self, message, reply_to=None):
        """Send a message to this actor, with an optional reply-to actor"""
        self.message_queue.put((message, reply_to))

    def process_messages(self):
        """Process messages sequentially from the message queue"""
        while True:
            message, reply_to = self.message_queue.get()
            if message == "STOP":
                break
            self.handle_message(message, reply_to)

    def handle_message(self, message, reply_to):
        """Handle the received message and send reply if necessary"""
        print(f"Actor {self.name} received message: {message}")
        time.sleep(1)  # Simulate some processing time

    def stop(self):
        """Stop the actor's message processing"""
        self.send_message("STOP", None)


class ActorSystem:
    def __init__(self, num_workers=3):  # Increased number of workers
        self.executor = ThreadPoolExecutor(max_workers=num_workers)
        self.actors = {}
    
    def create_actor(self, name):
        """Create an actor and run it"""
        actor = Actor(name, self)
        self.actors[name] = actor
        self.run_actor(actor)
        return actor
    
    def run_actor(self, actor):
        """Run an actor in the thread pool"""
        self.executor.submit(actor.process_messages)
    
    def send_message(self, target_actor_name, message, reply_to=None):
        """Send a message to a specific actor"""
        target_actor = self.actors.get(target_actor_name)
        if target_actor:
            target_actor.send_message(message, reply_to)

    def stop(self):
        """Stop the thread pool and all actors"""
        self.executor.shutdown(wait=True)


# Example Usage
if __name__ == "__main__":
    # Create an actor system with 3 threads (one for each actor)
    actor_system = ActorSystem(num_workers=3)
    
    # Create actors A, B, and C
    actor_a = actor_system.create_actor("A")
    actor_b = actor_system.create_actor("B")
    actor_c = actor_system.create_actor("C")
    
    # Send messages between actors
    actor_system.send_message("A", "Hello! from B", "B") # destination, message, sender
    actor_system.send_message("B", "Hello! from A", "A")
    
    # Simulate more complex interactions where A sends a message to B, and B replies back to A
    actor_system.send_message("B", "How are you? from A", "A")
    actor_system.send_message("A", "Doing well! from B", "B")
    
    # Introduce Actor C to send a message to Actor A
    actor_system.send_message("A", "How is everything? from C", "C")
    
    # Actor A can also send a message to Actor C
    actor_system.send_message("C", "Good! from A", "A")

    # Wait a bit to process the messages
    time.sleep(6)

    # Stop the actors gracefully
    actor_a.stop()
    actor_b.stop()
    actor_c.stop()

    # Shutdown the actor system (thread pool)
    actor_system.stop()


Actor B received message: Hello! from A
Actor A received message: Hello! from B
Actor C received message: Good! from A
Actor B received message: How are you? from A
Actor A received message: Doing well! from B
Actor A received message: How is everything? from C


### Actor system F2:
Within each actor, the received messages will be queued and sequentially processed. <br>
Each of the three actors will receive two greetings from the other two actors. <br>
It should be observed that the reply of greetings for each actor should be all after the send.

In [44]:
import queue
import time
from concurrent.futures import ThreadPoolExecutor

class Actor:
    def __init__(self, name, actor_system):
        self.name = name
        self.message_queue = queue.Queue()
        self.actor_system = actor_system  # To send messages to other actors

    def send_message(self, message, reply_to=None):
        """Send a message to this actor, with an optional reply-to actor"""
        self.message_queue.put((message, reply_to))
        # Print current message queue contents in FIFO order
        print(f"Actor {self.name}'s message queue: {[msg[0] for msg in list(self.message_queue.queue)]}")

    def process_messages(self):
        """Process messages sequentially from the message queue"""
        while True:
            message, reply_to = self.message_queue.get()  # Get message and reply_to info
            if message == "STOP":
                break
            self.handle_message(message, reply_to)

    def handle_message(self, message, reply_to):
        """Handle the received message and reply if needed based on message content"""
        print(f"Actor {self.name} processing message: '{message}'")
        time.sleep(1)  # Simulate processing time

    def stop(self):
        """Stop the actor's message processing"""
        self.message_queue.put(("STOP", None))


class ActorSystem:
    def __init__(self, num_workers=3):  # Number of worker threads
        self.executor = ThreadPoolExecutor(max_workers=num_workers)
        self.actors = {}

    def create_actor(self, name):
        """Create an actor and run it"""
        actor = Actor(name, self)
        self.actors[name] = actor
        self.run_actor(actor)
        return actor

    def run_actor(self, actor):
        """Run an actor in the thread pool"""
        self.executor.submit(actor.process_messages)

    def send_message(self, target_actor_name, message, reply_to=None):
        """Send a message to a specific actor"""
        target_actor = self.actors.get(target_actor_name)
        if target_actor:
            target_actor.send_message(message, reply_to)

    def stop(self):
        """Stop the thread pool and all actors"""
        self.executor.shutdown(wait=True)


# Example Usage
if __name__ == "__main__":
    # Create an actor system with 3 threads (one for each actor)
    actor_system = ActorSystem(num_workers=3)

    # Create actors A, B, and C
    actor_a = actor_system.create_actor("A")
    actor_b = actor_system.create_actor("B")
    actor_c = actor_system.create_actor("C")

    # Actor A asks
    actor_system.send_message("B", "How are you? from A", "A") 
    actor_system.send_message("C", "How are you? from A", "A") 
    
    # Actor B asks 
    actor_system.send_message("A", "Nice to meet you! from B", "B")
    actor_system.send_message("C", "Nice to meet you! from B", "B")

    # Actor C asks 
    actor_system.send_message("A", "How is everything? from C", "C")
    actor_system.send_message("B", "How is everything? from C", "C")

    # Wait a bit to process the messages
    time.sleep(5)

    # Stop the actors gracefully
    actor_a.stop()
    actor_b.stop()
    actor_c.stop()

    # Shutdown the actor system (thread pool)
    actor_system.stop()


Actor B's message queue: ['How are you? from A']
Actor C's message queue: ['How are you? from A']
Actor A's message queue: ['Nice to meet you! from B']
Actor C's message queue: ['How are you? from A', 'Nice to meet you! from B']
Actor A's message queue: ['Nice to meet you! from B', 'How is everything? from C']
Actor B's message queue: ['How are you? from A', 'How is everything? from C']
Actor B processing message: 'How are you? from A'
Actor C processing message: 'How are you? from A'Actor A processing message: 'Nice to meet you! from B'

Actor B processing message: 'How is everything? from C'
Actor C processing message: 'Nice to meet you! from B'
Actor A processing message: 'How is everything? from C'


### Actor system F3:
The state will be accessed and modified sequentially in actors. <br>
The actor B and actor C simultaneously sends one number (10 and 15) to the actor A, respectively. In the actor A, the two numbers will be both added to the inner state. <br>
It should be observed the state in the actor A should be 10 first then 25.

In [35]:
import queue
import time
from concurrent.futures import ThreadPoolExecutor

class Actor:
    def __init__(self, name, actor_system):
        self.name = name
        self.message_queue = queue.Queue()
        self.actor_system = actor_system  # To send messages to other actors
        self.state = 0  # Shared state, initialized to 0

    def send_message(self, message, reply_to=None):
        """Send a message to this actor, with an optional reply-to actor"""
        print(f"Message '{message}' enqueued for Actor {self.name}")
        self.message_queue.put((message, reply_to))

    def process_messages(self):
        """Process messages sequentially from the message queue"""
        while True:
            message, reply_to = self.message_queue.get()  # Get message and reply_to info
            if message == "STOP":
                print(f"Actor {self.name} is stopping.")
                break
            self.handle_message(message, reply_to)

    def handle_message(self, message, reply_to):
        """Handle the received message and update shared state"""
        print(f"Actor {self.name} received message: '{message}'")
        time.sleep(1)  # Simulate processing time

        print(f"Actor {self.name} is processing a numeric message: {message}")
        self.state += message
        print(f"Actor {self.name} state updated: {self.state}")

    def stop(self):
        """Stop the actor's message processing"""
        self.message_queue.put(("STOP", None))


class ActorSystem:
    def __init__(self, num_workers=3):  # Number of worker threads
        self.executor = ThreadPoolExecutor(max_workers=num_workers)
        self.actors = {}

    def create_actor(self, name):
        """Create an actor and run it"""
        actor = Actor(name, self)
        self.actors[name] = actor
        self.run_actor(actor)
        return actor

    def run_actor(self, actor):
        """Run an actor in the thread pool"""
        self.executor.submit(actor.process_messages)

    def send_message(self, target_actor_name, message, reply_to=None):
        """Send a message to a specific actor"""
        target_actor = self.actors.get(target_actor_name)
        if target_actor:
            print(f"ActorSystem sending message '{message}' from Actor {reply_to} to Actor {target_actor_name}")
            target_actor.send_message(message, reply_to)

    def stop(self):
        """Stop the thread pool and all actors, ensuring all messages are processed"""
        for actor in self.actors.values():
            actor.stop()
        self.executor.shutdown(wait=True)  # Wait for all threads to finish


# Example Usage
if __name__ == "__main__":
    # Create an actor system with 3 threads (one for each actor)
    actor_system = ActorSystem(num_workers=3)

    # Create actors A, B, and C
    actor_a = actor_system.create_actor("A")
    actor_b = actor_system.create_actor("B")
    actor_c = actor_system.create_actor("C")

    # Actor B sends messages with numbers to Actor A
    actor_system.send_message("A", 10, "B")  # Add 10 to the shared state of Actor A from Actor B

    # Actor C sends a message with a number to Actor A
    actor_system.send_message("A", 15, "C")  # Add 15 to the shared state of Actor A from Actor C

    # Wait a bit to process the messages
    time.sleep(5)  # Adjust as needed to ensure all messages are processed

    # Ensure actors are stopped after all messages are processed
    actor_system.stop() 

    # Shutdown the actor system (thread pool)
    print("Actor system stopped.")


ActorSystem sending message '10' from Actor B to Actor A
Message '10' enqueued for Actor A
ActorSystem sending message '15' from Actor C to Actor A
Message '15' enqueued for Actor A
Actor A received message: '10'
Actor A is processing a numeric message: 10
Actor A state updated: 10
Actor A received message: '15'
Actor A is processing a numeric message: 15
Actor A state updated: 25
Actor B is stopping.Actor A is stopping.Actor C is stopping.


Actor system stopped.


### Actor system F41:
The message quque can be set to be processed with priority. <br>
Continue with the last example, the actor A receives the number 10 first then 15. <br>
However, we want to observe the state is added to be first 15 then 25.

In [50]:
import queue
import time
from concurrent.futures import ThreadPoolExecutor

class Actor:
    def __init__(self, name, actor_system):
        self.name = name
        self.message_queue = queue.PriorityQueue()  # Priority Queue for messages
        self.actor_system = actor_system  # To send messages to other actors
        self.state = 0  # Shared state, initialized to 0

    def send_message(self, priority, message, reply_to=None):
        """Send a message with a priority to this actor, with an optional reply-to actor"""
        print(f"Message '{message}' with priority '{priority}' enqueued for Actor {self.name}")
        self.message_queue.put((priority, message, reply_to))  # Priority, message, and reply_to

    def process_messages(self):
        """Process messages sequentially from the message queue"""
        while True:
            priority, message, reply_to = self.message_queue.get()  # Get message, priority, and reply_to info
            if message == "STOP":
                print(f"Actor {self.name} is stopping.")
                break
            self.handle_message(priority, message, reply_to)

    def handle_message(self, priority, message, reply_to):
        """Handle the received message and update shared state"""
        print(f"Actor {self.name} received message: '{message}' with priority '{priority}'")
        time.sleep(1)  # Simulate processing time

        self.state += message
        print(f"Actor {self.name} state updated: {self.state}")

    def stop(self):
        """Stop the actor's message processing"""
        self.message_queue.put((0, "STOP", None))


class ActorSystem:
    def __init__(self, num_workers=3):  # Number of worker threads
        self.executor = ThreadPoolExecutor(max_workers=num_workers)
        self.actors = {}

    def create_actor(self, name):
        """Create an actor and run it"""
        actor = Actor(name, self)
        self.actors[name] = actor
        self.run_actor(actor)
        return actor

    def run_actor(self, actor):
        """Run an actor in the thread pool"""
        self.executor.submit(actor.process_messages)

    def send_message(self, target_actor_name, priority, message, reply_to=None):
        """Send a message with a priority to a specific actor"""
        target_actor = self.actors.get(target_actor_name)
        if target_actor:
            print(f"ActorSystem sending message '{message}' with priority '{priority}' from Actor {reply_to} to Actor {target_actor_name}")
            target_actor.send_message(priority, message, reply_to)

    def stop(self):
        """Stop the thread pool and all actors, ensuring all messages are processed"""
        for actor in self.actors.values():
            actor.stop()
        self.executor.shutdown(wait=True)  # Wait for all threads to finish


# Example Usage
if __name__ == "__main__":
    # Create an actor system with 3 threads (one for each actor)
    actor_system = ActorSystem(num_workers=3)

    # Create actors A, B, and C
    actor_a = actor_system.create_actor("A")
    actor_b = actor_system.create_actor("B")
    actor_c = actor_system.create_actor("C")

    # Actor B sends messages with different priorities to Actor A
    actor_system.send_message("A", 2, 10, "B")  # Lower priority for increment by 10
    actor_system.send_message("A", 1, 15, "C")  # Higher priority for increment by 15

    # Wait a bit to process the messages
    time.sleep(5)  # Adjust as needed to ensure all messages are processed

    # Stop actors A, B, and C after confirming all messages are processed
    actor_system.send_message("A", 0, "STOP")  # Stop Actor A last after all messages are processed
    time.sleep(2)  # Give enough time to process the stop message

    # Ensure actors are stopped after all messages are processed
    actor_system.stop()

    # Shutdown the actor system (thread pool)
    print("Actor system stopped.")


ActorSystem sending message '10' with priority '2' from Actor B to Actor A
Message '10' with priority '2' enqueued for Actor A
ActorSystem sending message '15' with priority '1' from Actor C to Actor A
Message '15' with priority '1' enqueued for Actor A
Actor A received message: '15' with priority '1'
Actor A state updated: 15
Actor A received message: '10' with priority '2'
Actor A state updated: 25
ActorSystem sending message 'STOP' with priority '0' from Actor None to Actor A
Message 'STOP' with priority '0' enqueued for Actor A
Actor A is stopping.
Actor B is stopping.
Actor C is stopping.
Actor system stopped.


### Actor system F42:
The messages in the message queue (in FIFO), which cannot be processed now, can be requeued to the end of the message queue for later processing. <br>
The below example works fundamentally similar to **controller** in cell search. <br>
The actor B and C send 0,1,2 three states to actor A. <br>
The initial state of actor A is 0. The rule of state transition is performed as follows:
- when state is 0, performs message of state 2. The inner state then switches to 2.
- when state is 2, performs message of state 1. The inner state then switches to 1.
- when state is 1, performs message of state 0. The inner state then switches to 0.

Otherwise, the message will be re-queued to the end of the message queue.

In [54]:
import queue
import time
from concurrent.futures import ThreadPoolExecutor

class Actor:
    def __init__(self, name, actor_system):
        self.name = name
        self.message_queue = queue.Queue()
        self.actor_system = actor_system  # To send messages to other actors
        self.state = 0  # Shared state, initialized to 0

    def send_message(self, message, reply_to=None):
        """Send a message to this actor, with an optional reply-to actor"""
        print(f"Message '{message}' enqueued for Actor {self.name}")
        self.message_queue.put((message, reply_to))

    def process_messages(self):
        """Process messages sequentially from the message queue"""
        while True:
            message, reply_to = self.message_queue.get()  # Get message and reply_to info
            if message == "STOP":
                print(f"Actor {self.name} is stopping.")
                break
            self.handle_message(message, reply_to)

    def handle_message(self, message, reply_to):
        """Handle the received message and update shared state based on specified rules"""
        print(f"Actor {self.name} received message: '{message}'")
        time.sleep(1)  # Simulate processing time

        # Only process messages based on the inner state
        if self.state == 0 and message == 2:
            print(f"Actor {self.name} processing message: {message}")
            self.state = 2
        elif self.state == 2 and message == 1:
            print(f"Actor {self.name} processing message: {message}")
            self.state = 1
        elif self.state == 1 and message == 0:
            print(f"Actor {self.name} processing message: {message}")
            self.state = 0
        else:
            # If the message isn't valid for the current state, requeue it
            print(f"Actor {self.name} requeuing message: {message} due to current state: {self.state}")
            self.message_queue.put((message, reply_to))  # Requeue the message

    def stop(self):
        """Stop the actor's message processing"""
        self.message_queue.put(("STOP", None))


class ActorSystem:
    def __init__(self, num_workers=3):  # Number of worker threads
        self.executor = ThreadPoolExecutor(max_workers=num_workers)
        self.actors = {}

    def create_actor(self, name):
        """Create an actor and run it"""
        actor = Actor(name, self)
        self.actors[name] = actor
        self.run_actor(actor)
        return actor

    def run_actor(self, actor):
        """Run an actor in the thread pool"""
        self.executor.submit(actor.process_messages)

    def send_message(self, target_actor_name, message, reply_to=None):
        """Send a message to a specific actor"""
        target_actor = self.actors.get(target_actor_name)
        if target_actor:
            print(f"ActorSystem sending message '{message}' from Actor {reply_to} to Actor {target_actor_name}")
            target_actor.send_message(message, reply_to)

    def stop(self):
        """Stop the thread pool and all actors, ensuring all messages are processed"""
        for actor in self.actors.values():
            actor.stop()
        self.executor.shutdown(wait=True)  # Wait for all threads to finish


# Example Usage
if __name__ == "__main__":
    # Create an actor system with 3 threads (one for each actor)
    actor_system = ActorSystem(num_workers=3)

    # Create actors A, B, and C
    actor_a = actor_system.create_actor("A")
    actor_b = actor_system.create_actor("B")
    actor_c = actor_system.create_actor("C")

    # Actors B and C send messages with numbers to Actor A
    actor_system.send_message("A", 0, "B")  # Process 2 when state is 0
    actor_system.send_message("A", 1, "C")  # Will be requeued
    actor_system.send_message("A", 2, "B")  # Will be requeued

    # Wait a bit to process the messages
    time.sleep(10)  # Adjust as needed to ensure all messages are processed

    # Ensure actors are stopped after all messages are processed
    actor_system.stop()

    # Shutdown the actor system (thread pool)
    print("Actor system stopped.")


ActorSystem sending message '0' from Actor B to Actor A
Message '0' enqueued for Actor A
ActorSystem sending message '1' from Actor C to Actor A
Message '1' enqueued for Actor A
ActorSystem sending message '2' from Actor B to Actor A
Message '2' enqueued for Actor A
Actor A received message: '0'
Actor A requeuing message: 0 due to current state: 0
Actor A received message: '1'
Actor A requeuing message: 1 due to current state: 0
Actor A received message: '2'
Actor A processing message: 2
Actor A received message: '0'
Actor A requeuing message: 0 due to current state: 2
Actor A received message: '1'
Actor A processing message: 1
Actor A received message: '0'
Actor A processing message: 0
Actor A is stopping.
Actor B is stopping.
Actor C is stopping.
Actor system stopped.


### Task parallelism inside actors
We want actor system to dispatch the right task to the right processing pipeline based on the system's state. <br>
Within each actor, we want the task can be executed effectively by task-oriented programming, e.g., TBB. <br>
In the below example, we compute a square (pure) function within an actor in a parallel way to show multithreaded computation can be performed inside an actor. After that, the vector of squared values will be accumulated to the inner state of the actor. <br>
It should be observed that on the top level, the actor processes the received message one at the time (with order) and for each message, multiple threads are simultaneously used to compute a square (pure) function. Furthermore, the state should be accumulated correctly.

In [55]:
import threading
from concurrent.futures import ThreadPoolExecutor
import time
import queue
import os

# Base Actor class for message-passing and sequential message processing
class Actor:
    def __init__(self):
        self.message_queue = queue.Queue()

    def send_message(self, message):
        self.message_queue.put(message)

    def stop(self):
        self.message_queue.put("STOP")

# WorkerActor uses TBB-like parallelism for task processing
class WorkerActor(Actor):
    def __init__(self, name):
        super().__init__()
        self.name = name
        self.executor = ThreadPoolExecutor(max_workers=4)  # Simulating TBB with ThreadPool
        self.state = 0  # Shared state, but only updated sequentially

    def process_task(self, data):
        # Simulate parallel processing and print thread ID to show concurrent execution
        def task(x):
            print(f"Processing {x} on thread {threading.get_ident()}")
            time.sleep(1)  # Simulate work delay
            return x * x

        results = list(self.executor.map(task, data))  # Parallel computation
        print(f"{self.name} processed data {data} -> {results}")

        # After parallel processing, update state sequentially
        self.update_state(sum(results))

    def update_state(self, result_sum):
        # Sequentially update shared state (no locks needed)
        self.state += result_sum
        print(f"{self.name} updated state to: {self.state}")

    def process_messages(self):
        """Actor's thread processes messages sequentially"""
        while True:
            message = self.message_queue.get()
            if message == "STOP":
                print(f"Worker {self.name} is stopping.")
                break
            self.process_task(message)  # Hand off task to TBB for parallel execution

# PriorActor that sends tasks to WorkerActor
class PriorActor(Actor):
    def __init__(self, worker_actor):
        super().__init__()
        self.worker_actor = worker_actor

    def send_task_to_worker(self, task):
        print(f"PriorActor is sending task: {task} to WorkerActor")
        self.worker_actor.send_message(task)

    def process_messages(self):
        """Process messages sent to PriorActor if needed (not used in this example)"""
        pass

# Example of how the system works
if __name__ == "__main__":
    # Create WorkerActor
    worker_actor = WorkerActor("WorkerActor")

    # Create PriorActor that sends tasks to WorkerActor
    prior_actor = PriorActor(worker_actor)

    # Start worker actor in a separate thread (actor's thread)
    threading.Thread(target=worker_actor.process_messages).start()

    # PriorActor sends tasks to WorkerActor
    prior_actor.send_task_to_worker([1, 2, 3, 4])
    prior_actor.send_task_to_worker([5, 6, 7, 8])

    # Allow time for processing
    time.sleep(5)

    # Stop the worker actor
    worker_actor.stop()


PriorActor is sending task: [1, 2, 3, 4] to WorkerActor
PriorActor is sending task: [5, 6, 7, 8] to WorkerActor
Processing 1 on thread 140229292787456
Processing 2 on thread 140228937381632
Processing 3 on thread 140228928988928
Processing 4 on thread 140228920596224
WorkerActor processed data [1, 2, 3, 4] -> [1, 4, 9, 16]
WorkerActor updated state to: 30
Processing 5 on thread 140229292787456
Processing 6 on thread 140228937381632
Processing 7 on thread 140228928988928
Processing 8 on thread 140228920596224
WorkerActor processed data [5, 6, 7, 8] -> [25, 36, 49, 64]
WorkerActor updated state to: 204
Worker WorkerActor is stopping.
